# WEEK 3: GCP Service

## What is GCP Services
Google Cloud Platform (GCP) services are cloud-based tools and APIs that let applications automatically perform tasks like uploading files, processing data, accessing storage, or interacting with other Google services such as Gmail, Drive, and BigQuery.

In [ ]:
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

## Google Sheets

Task: 
- read data
- add new column
- add new row
- add new sheet

https://docs.google.com/spreadsheets/d/1Kz4OJyG_VpLd5_4XcorN3WzdS32KFU9N7RsQo_I92Cs/edit?usp=sharing

In [ ]:
import os
import time
import pickle

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
CREDENTIALS_FILE = "credentials.json"
TOKEN_PICKLE = "token.pickle"
SPREADSHEET_ID = "1Kz4OJyG_VpLd5_4XcorN3WzdS32KFU9N7RsQo_I92Cs"

creds = None

if os.path.exists(TOKEN_PICKLE):
    with open(TOKEN_PICKLE, "rb") as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    with open(TOKEN_PICKLE, "wb") as token:
        pickle.dump(creds, token)

sheets_service = build("sheets", "v4", credentials=creds)

values = sheets_service.spreadsheets().values().get(
    spreadsheetId=SPREADSHEET_ID,
    range="movies"
).execute().get("values", [])

header = values[0]
data_rows = values[1:]
output_rows = []

options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled") #menambahkan blink
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => false});") #pakai false dibanding undefined
wait = WebDriverWait(driver, 20)

for row_values in data_rows:
    row = {header[i]: row_values[i] if i < len(row_values) else "" for i in range(len(header))}
    url = row.get("url", "").strip()
    if not url:
        row["director"] = row["writer"] = row["star"] = ""
        output_rows.append(row)
        continue
    driver.get(url)

    wait.until(EC.presence_of_element_located((
        By.XPATH,
        "//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section" #revisi xpath
    )))

    def ambil(idx):
        path = (f"//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/"
                f"div[3]/div[2]/div[1]/section/div[2]/ul/li[{idx}]/div/ul//li") #revisi xpath
        return ", ".join(el.text.strip() for el in driver.find_elements(By.XPATH, path) if el.text.strip())

    row["director"] = ambil(1)
    row["writer"]  = ambil(2)
    row["star"]    = ambil(3)

    output_rows.append(row)
    time.sleep(5)

driver.quit()

output_header = header[:] #perubahan struktur syntax biar lebih flexible
for col in ["director", "writer", "star"]:
    if col not in output_header:
        output_header.append(col)

output_values = [output_header] + [
    [row.get(col, "") for col in output_header] for row in output_rows
]

sheet_meta = sheets_service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID
).execute()

sheet_titles = [s["properties"]["title"] for s in sheet_meta.get("sheets", [])]

if "movies_detailed" not in sheet_titles:
    sheets_service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={"requests": [{"addSheet": {"properties": {"title": "movies_detailed"}}}]}
    ).execute()
else:
    sheets_service.spreadsheets().values().clear(
        spreadsheetId=SPREADSHEET_ID,
        range="movies_detailed!A:Z" #spesifik kan range yang di clear
    ).execute()

sheets_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range="movies_detailed!A1", #spesifik kan range awal
    valueInputOption="RAW",
    body={"values": output_values}
).execute()

{'spreadsheetId': '1Kz4OJyG_VpLd5_4XcorN3WzdS32KFU9N7RsQo_I92Cs',
 'updatedRange': 'movies_data!A1:D4',
 'updatedRows': 4,
 'updatedColumns': 4,
 'updatedCells': 16}

## Google Drive

Task: 
- download
- upload

Pre-Task:
- Add new OAuth 2.0 Client IDs
- Oauth consent screen -> audience -> add users